In [ ]:
# default_exp daemon

# pico_pi_controller.daemon
> System daemon

In [ ]:
# hide
# from nbdev.showdoc import *

In [ ]:
from pico_pi_controller.core import *

In [ ]:
# export

def i2c_event_handler(id, tick):
    """Copied from local ppdd.py."""
    global pi
    global i2c_addr
    global flicker

    def regIDF():
        print ("IDF recieved, sending identity:",IDENTITY.decode(),"on tick",tick)
        s, b, d = pi.bsc_i2c(i2c_addr,IDENTITY)
        
    def regTIM():
        dt = datetime.now()
        dt=dt.replace(tzinfo=timezone(timedelta(0),'UTC')) #send local time converted from UTC of system time
        ts=dt.timestamp()
        print ("TIM recieved, sending bytes len, time.time():",len((bytearray(int(ts).to_bytes(4,byteorder)))), int(ts))
        s, b, d = pi.bsc_i2c(i2c_addr,bytearray(int(ts).to_bytes(4,byteorder)))

    def regBOS():
        print ("BOS recieved, sending bosmang status:",bosmang.decode())
        s, b, d = pi.bsc_i2c(i2c_addr,bosmang)
        
    def regCMD():
            print ("CMD recieved, sending queud command code (or 0 if none):",cmd_code.decode())
            s, b, d = pi.bsc_i2c(i2c_addr,cmd_code)
            
    def regHOS():
        print ("HOS recieved, sending hostname:",hostname.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, bytes([len(hostname)])+hostname)

    def regLOD():
        load = bytearray("{:04.2f}".format(os.getloadavg()[0]),'utf-8')
        print ("LOD recieved, sending load:",load.decode())
        s, b, d = pi.bsc_i2c(i2c_addr,load)

    def regUPT():
        # so the PPC can pause and wait for the long-ish uptime fetch
        s, b, d = pi.bsc_i2c(i2c_addr,pause)
        uptimeba = bytearray(int(uptime()).to_bytes(4,byteorder))
        print ("UPT recieved, sending uptime:",int.from_bytes(bytes(uptimeba),byteorder))
        s, b, d = pi.bsc_i2c(i2c_addr,uptimeba)
    
    def regTZN():
        print ("TZN recieved, sending bytes len, time.timezone():",
            len((bytearray(time.timezone.to_bytes(3,byteorder)))), time.timezone)
        s, b, d = pi.bsc_i2c(i2c_addr,bytearray(time.timezone.to_bytes(3,byteorder)))

    def regFLK():
        print ("FLK recieved, Flickering PWR LED for",d[1],"seconds.")
        flicker = PwrLedFlicker(duration=d[1])
        flicker.start()

    def regCLR():
        print ("CLR recieved, sending empty message.")
        s, b, d = pi.bsc_i2c(i2c_addr,empty)
           
    s, b, d = pi.bsc_i2c(i2c_address)
    if b and d[0] in REG_INT:
        reg_handler = getattr(i2c_event_handler, 'reg'+REG_NAME[d[0]])
        reg_handler()


def main():
    global pi
    global flicker
               
    os.environ['TZ']="America/New_York"
    time.tzset()

    @atexit.register
    def goodbye():
        """Cancel pigpio event handler, close I2C peripheral & connection to pigpio"""
        try:
            e.cancel()
            pi.bsc_i2c(0) # Disable BSC peripheral
            pi.stop()
            print ("bsc_i2c cleanup completed.")
        except:
            print ("bsc_i2c error on cleanup.")
        print ('Exiting via atexit')
               
    flicker = PwrLedFlicker(duration=1)
    flicker.start()
    print("PP Device daemon: Initializing I2C peripheral on address",hex(I2C_ADDR))
    print("PP Device daemon: bosmang:",bool(bosmang.decode()))
    print("PP Device daemon: hostname:",hostname.decode())
                                                                         
    pi = pigpio.pi()
    if not pi.connected:
        print ("pigpiod not running. Exiting.")
        exit()
                                                                         
    # setup event handler for incoming I2C messages from PPController
    e = pi.event_callback(pigpio.EVENT_BSC, self.i2c_event_handler)
    pi.bsc_i2c(i2c_addr)                                  
                                                                         
    while True:
        #here goes the socket connection for ppcc & listening for system events
        time.sleep(600)

In [ ]:
# hide
try:
    from IPython.display import display, Javascript
    display(Javascript('IPython.notebook.save_checkpoint();'))
    from time import sleep
    sleep(0.3)
    from nbdev.export import notebook2script
    notebook2script()
except ModuleNotFoundError:
    pass
    """CircuitPython kernel has no nbdev"""

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 50_daemon.ipynb.
Converted index.ipynb.
